In [2]:
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from qdrant_client import QdrantClient
from qdrant_client import models
import numpy as np


client = QdrantClient(url="http://10.11.12.134:6333", timeout=20)
#client = QdrantClient("http://localhost", port=6333, grpc_port=6334)

COLLECTION_NAME = 'wikipedia_20231101_en_text512_embeddings'

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-large',
    model_kwargs={"device": "cpu"},
    cache_folder='cache'
)

/tmp/ipykernel_69208/3294939613.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(


In [29]:
import os
import json
import pandas as pd

DATA_DIR = '/data/semeval2025/data'
# Чтение всех .jsonl файлов из директории
all_data = []
for file_name in os.listdir(DATA_DIR):
    if file_name.endswith('.jsonl'):
        file_path = os.path.join(DATA_DIR, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                all_data.append(json.loads(line))

# Построение DataFrame
df = pd.DataFrame(all_data)
df

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens
0,val-hi-1,HI,वाइट कॉलर टीवी शृंख्ला में मुख़्य किरदार को जेल...,सजायाफ्ता कैदियों को टेलीविज़न (टेलीविज़न) सीरीज़...,nickmalhotra/ProjectIndus,"[{'start': 0, 'prob': 0.6666666667, 'end': 17}...","[[0, 17], [48, 59], [142, 286], [374, 520], [5...","[7.3444852829, 9.7444591522, 20.5029029846, 20...","[स, जाया, फ्, ता, ▁कै, दिय, ों▁को▁, टेलीवि, ज़,..."
1,val-hi-2,HI,गलिशिया यूरोप के किस देश का हिस्सा है ?,"घलंडिया के पास स्वीडन, कोरिया और मेक्सिको के स...",nickmalhotra/ProjectIndus,"[{'start': 0, 'prob': 0.6666666667, 'end': 15}...","[[0, 115]]","[9.5148887634, 12.2905397415, 13.2247562408, 1...","[घ, ल, ंड, िया▁के▁, पास, ▁स्वी, ड, न, ,▁को, रि..."
2,val-hi-3,HI,मीर अंतरिक्ष स्टेशन का प्रक्षेपण किस देश ने क...,मीर अंतरिक्ष स्टेशन का प्रक्षेपण चीन ने किया है।,nickmalhotra/ProjectIndus,"[{'start': 33, 'prob': 1.0, 'end': 36}]","[[33, 36]]","[13.5621585846, 19.3110408783, 21.622253418, 1...","[मीर▁, अंतरिक्ष, ▁स्टेशन, ▁का▁प्र, क्षे, पण▁, ..."
3,val-hi-4,HI,डवे फ्रेंको अपने जीवनसाथी से मिलने के कितने सम...,डवे फ्रेंको अपने जीवनसाथी से मिलने के कितने सम...,sarvamai/OpenHathi-7B-Hi-v0.1-Base,"[{'start': 77, 'prob': 0.3333333333, 'end': 86...","[[90, 94]]","[14.4820013046, 21.4985790253, 21.3416728973, ...","[Answer, :, <0x0A>, ---, <0x0A>, ड, व, ▁फ्र, े..."
4,val-hi-5,HI,सामान्य मानव जन्म में पूरे कितने चरण होते है ?,सामान्य मानव जन्म में पूरे एक चरण होते हैं ।,nickmalhotra/ProjectIndus,"[{'start': 27, 'prob': 1.0, 'end': 29}]","[[27, 29]]","[13.0817899704, 18.6904373169, 19.4961681366, ...","[सामान्य▁, मानव▁, जन्, म▁में▁, पूरे▁, एक▁, चरण..."
...,...,...,...,...,...,...,...,...,...
494,val-it-46,IT,Chi doppia in italiano la voce del personaggio...,La voce italiana del personaggio televisivo Su...,sapienzanlp/modello-italia-9b,"[{'start': 85, 'prob': 1.0, 'end': 98}]","[[85, 98]]","[128.0, 264.0, 316.0, 306.0, 284.0, 248.0, 199...","[La, ▁voce, ▁italiana, ▁del, ▁personaggio, ▁te..."
495,val-it-47,IT,A che altitudine si trova Sagar capoluogo del ...,"Sagar, il capoluogo del distretto di Sagar, si...",Qwen/Qwen2-7B-Instruct,"[{'start': 79, 'prob': 1.0, 'end': 82}]","[[79, 82]]","[24.375, 23.625, 29.125, 22.375, 22.625, 30.25...","[S, agar, ,, Ġil, Ġcap, olu, ogo, Ġdel, Ġdist,..."
496,val-it-48,IT,Quali stati attraversa il fiume Irrawaddy?,Il fiume Irrawaddy attraversa diversi paesi in...,sapienzanlp/modello-italia-9b,"[{'start': 30, 'prob': 0.3333333333, 'end': 37...","[[53, 87], [125, 168]]","[225.0, 272.0, 250.0, 181.0, 163.0, 174.0, 262...","[Il, ▁fiume, ▁Ir, raw, add, y, ▁attraversa, ▁d..."
497,val-it-49,IT,Quali sono le specie in cui possono presentars...,Le nubi cirrostrato possono presentarsi in div...,sapienzanlp/modello-italia-9b,"[{'start': 0, 'prob': 0.3333333333, 'end': 43}...","[[43, 50], [67, 418]]","[172.0, 193.0, 246.0, 192.0, 191.0, 253.0, 156...","[Le, ▁nubi, ▁cir, ro, strato, ▁possono, ▁prese..."


In [30]:
# Добавление колонок doc1, ..., doc5
j = 0
#df = df[:10]
def fetch_top_docs(query):
    """Получает топ-5 документов из векторной БД."""
    global j
    embedding = embeddings.embed_query(f"query: {query}")
    results = client.query_points(
        collection_name=COLLECTION_NAME,
        query=embedding,
        limit=5,
        with_vectors=False,
        timeout=300
    )
    j += 1
    response = [
        {
            'id': result.id,
            'score': result.score,
            'text': result.payload['text'],
            'title': result.payload['my_metadata']['title'],
            'url': result.payload['my_metadata']['source']
        }
        for result in results.points
    ]
    print(j, len(results.points))
    return response  # Предполагается, что ID документов возвращаются в поле 'id'

# Добавляем новые колонки в DataFrame
for i in range(1, 6):
    df[f'doc{i}'] = df['model_input'].apply(lambda x: fetch_top_docs(x)[i-1])

# Сохраняем результат (если нужно)
output_path = '/data/semeval2025/semeval_val_RAG.json'
df.to_json(output_path)

1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
10 5
11 5
12 5
13 5
14 5
15 5
16 5
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
30 5
31 5
32 5
33 5
34 5
35 5
36 5
37 5
38 5
39 5
40 5
41 5
42 5
43 5
44 5
45 5
46 5
47 5
48 5
49 5
50 5
51 5
52 5
53 5
54 5
55 5
56 5
57 5
58 5
59 5
60 5
61 5
62 5
63 5
64 5
65 5
66 5
67 5
68 5
69 5
70 5
71 5
72 5
73 5
74 5
75 5
76 5
77 5
78 5
79 5
80 5
81 5
82 5
83 5
84 5
85 5
86 5
87 5
88 5
89 5
90 5
91 5
92 5
93 5
94 5
95 5
96 5
97 5
98 5
99 5
100 5
101 5
102 5
103 5
104 5
105 5
106 5
107 5
108 5
109 5
110 5
111 5
112 5
113 5
114 5
115 5
116 5
117 5
118 5
119 5
120 5
121 5
122 5
123 5
124 5
125 5
126 5
127 5
128 5
129 5
130 5
131 5
132 5
133 5
134 5
135 5
136 5
137 5
138 5
139 5
140 5
141 5
142 5
143 5
144 5
145 5
146 5
147 5
148 5
149 5
150 5
151 5
152 5
153 5
154 5
155 5
156 5
157 5
158 5
159 5
160 5
161 5
162 5
163 5
164 5
165 5
166 5
167 5
168 5
169 5
170 5
171 5
172 5
173 5
174 5
175 5
176 5
177 5
178 5
179 5
180 5
181 5
182 5
183 5
184 5
185 

In [26]:
df = pd.read_json('/data/semeval2025/semeval_val_RAG.json')

In [27]:
df

,id,lang,model_input,model_output_text,model_id,soft_labels,hard_labels,model_output_logits,model_output_tokens,doc1,doc2,doc3,doc4,doc5
0,val-hi-1,HI,वाइट कॉलर टीवी शृंख्ला में मुख़्य किरदार को जेल...,सजायाफ्ता कैदियों को टेलीविज़न (टेलीविज़न) सीरीज़...,nickmalhotra/ProjectIndus,"[{'start': 0, 'prob': 0.6666666667000001, 'end...","[[0, 17], [48, 59], [142, 286], [374, 520], [5...","[7.3444852829, 9.7444591522, 20.5029029846, 20...","[स, जाया, फ्, ता, ▁कै, दिय, ों▁को▁, टेलीवि, ज़,...","{'id': 10023401520, 'score': 0.7844734, 'text'...","{'id': 10009961109, 'score': 0.78147507, 'text...","{'id': 10025137911, 'score': 0.78063774, 'text...","{'id': 10063049560, 'score': 0.7803726, 'text'...","{'id': 10002529484, 'score': 0.77929115, 'text..."
1,val-hi-2,HI,गलिशिया यूरोप के किस देश का हिस्सा है ?,"घलंडिया के पास स्वीडन, कोरिया और मेक्सिको के स...",nickmalhotra/ProjectIndus,"[{'start': 0, 'prob': 0.6666666667000001, 'end...","[[0, 115]]","[9.5148887634, 12.2905397415, 13.2247562408, 1...","[घ, ल, ंड, िया▁के▁, पास, ▁स्वी, ड, न, ,▁को, रि...","{'id': 10000075205, 'score': 0.791141499999999...","{'id': 10059762804, 'score': 0.782409670000000...","{'id': 10019648746, 'score': 0.7810402, 'text'...","{'id': 10000336341, 'score': 0.780075099999999...","{'id': 10000298622, 'score': 0.7794876, 'text'..."
2,val-hi-3,HI,मीर अंतरिक्ष स्टेशन का प्रक्षेपण किस देश ने क...,मीर अंतरिक्ष स्टेशन का प्रक्षेपण चीन ने किया है।,nickmalhotra/ProjectIndus,"[{'start': 33, 'prob': 1.0, 'end': 36}]","[[33, 36]]","[13.5621585846, 19.3110408783, 21.622253418, 1...","[मीर▁, अंतरिक्ष, ▁स्टेशन, ▁का▁प्र, क्षे, पण▁, ...","{'id': 10000081326, 'score': 0.816989899999999...","{'id': 10000972968, 'score': 0.8137512, 'text'...","{'id': 10010473429, 'score': 0.81295776, 'text...","{'id': 10001906524, 'score': 0.8114433, 'text'...","{'id': 10029515652, 'score': 0.7998295, 'text'..."
3,val-hi-4,HI,डवे फ्रेंको अपने जीवनसाथी से मिलने के कितने सम...,डवे फ्रेंको अपने जीवनसाथी से मिलने के कितने सम...,sarvamai/OpenHathi-7B-Hi-v0.1-Base,"[{'start': 77, 'prob': 0.33333333330000003, 'e...","[[90, 94]]","[14.4820013046, 21.4985790253, 21.3416728973, ...","[Answer, :, <0x0A>, ---, <0x0A>, ड, व, ▁फ्र, े...","{'id': 10001423846, 'score': 0.760206199999999...","{'id': 10045711572, 'score': 0.758384699999999...","{'id': 10003395244, 'score': 0.75462914, 'text...","{'id': 10025851365, 'score': 0.7545185, 'text'...","{'id': 10049383809, 'score': 0.75060654, 'text..."
4,val-hi-5,HI,सामान्य मानव जन्म में पूरे कितने चरण होते है ?,सामान्य मानव जन्म में पूरे एक चरण होते हैं ।,nickmalhotra/ProjectIndus,"[{'start': 27, 'prob': 1.0, 'end': 29}]","[[27, 29]]","[13.0817899704, 18.6904373169, 19.4961681366, ...","[सामान्य▁, मानव▁, जन्, म▁में▁, पूरे▁, एक▁, चरण...","{'id': 10000083449, 'score': 0.8290405, 'text'...","{'id': 10004758634, 'score': 0.8244724, 'text'...","{'id': 10002190913, 'score': 0.813190460000000...","{'id': 10012276398, 'score': 0.81194687, 'text...","{'id': 10008904727, 'score': 0.81023407, 'text..."
5,val-hi-6,HI,शृंगिका का उपयोग किन किन कार्यो के लिए होता है ?,शृंगिका का उपयोग मुख्य रूप से संगीत में किया ज...,nickmalhotra/ProjectIndus,"[{'start': 0, 'prob': 0.5, 'end': 30}, {'start...","[[30, 35], [54, 2206]]","[15.5833873749, 22.3139324188, 21.5522785187, ...","[शृ, ंग, िका, ▁का▁उपयोग▁, मुख्य▁रूप▁से▁, संगीत...","{'id': 10023737925, 'score': 0.791011799999999...","{'id': 10017340059, 'score': 0.788282399999999...","{'id': 10000335129, 'score': 0.7875004, 'text'...","{'id': 10025773118, 'score': 0.782238, 'text':...","{'id': 10011250668, 'score': 0.78196144, 'text..."
6,val-hi-7,HI,प्रमस्तिष्क अंगघात बीमारी को पहली बार किस चिकि...,प्रमस्तिष्क अंगघात बीमारी को पहली बार १८८२ में...,nickmalhotra/ProjectIndus,"[{'start': 38, 'prob': 1.0, 'end': 42}, {'star...","[[38, 42], [47, 59], [95, 194]]","[12.9719810486, 21.6390380859, 18.3986110687, ...","[प्र, मस्तिष्क▁, अंग, घात▁, बीमारी, ▁को, ▁पहली...","{'id': 10002583562, 'score': 0.80249596, 'text...","{'id': 10027313858, 'score': 0.80093193000000